#                          Mustafa Can Coşkun - IE 203 - Homework 2

* In this homework, I will provide two distinct ways of obtaining steady state distribution of Ergodic Markov Chain.

1. Matrix Multiplication Method 
2. Monte Carlo Random Walk Simulation

Let me explain formulation of these methods.

### Matrix Multiplication Method

In Matrix Multiplication Method, I am multiplying the Transition Probability Matrix by itself until each column has the same value with the error bound of epsilon. Error bound is given explicitly. 
In order to find the value that will be compared with epsilon, I am choosing one column randomly with "random.randint()" method and taking the average of this column. Then, again, I pick a random row from chosen column and subtracting it from the average value. If the absolute value of this difference is smaller than the epsilon, I break the loop. If not, the while loop iterates up until the difference is smaller than the epsilon. 

### Monte Carlo Random Walk Simulation

Before starting this method, we need an external data set to keep record of the passed states. After defining it, we pick a random row, which is also initial state to start with by "random.randint()". Let's call it X_i. After picking an initial row, we randomly select a number between 0 and 1 in order to select the subsequent step to go, let's call it p. Then for every entry of row, we check if the p is between cumulative sum of entry's up until to p, not including p, and including p. When we find the true interval, our new X_i becomes the index of this true interval and we increase the number of pass times of specified state in our external data set. We repeat this procedure with explicitly specified number of steps. At the end, this function returns the initially defined data set with all the entries are divided with number of steps.

### Absorbing State Included

After calculating the steady state distribution of the markov chain with procedures given above, absorbing row is replaced in the place of a randomly selected state which can be observed in "absorbing_includer()" method. This row is obtained by choosing a random state and creating an array of all zeros except the index of randomly selected number. Then all process is repeated with above procedures.


## Now Let's Go to Python Implementation of This Process

### Importing libraries of numpy, random, time and IPython.

In [2]:
import numpy as np
import pandas as pd
import random
import time
from IPython.display import display

### Creating the Transition Probability Matrix and implementing the multplication and Monte Carlo Simulation method.

In [3]:
class TPM_Matrix(): #Class Representing Transition Probability Matrix and Related Operations

    def __init__(self,size,epsilon = 0.0005, step_size = 200000):
        self._size = size 
        self._matrix  = [] #Probability Transition Matrix
        self._MC_Visits = {f"State_{i}": 0 for i in  range(self._size)} #Keeping Record of Number of Visits in Monte Carlo Method
        self._epsilon = epsilon
        self._step_size = step_size

    def generate_matrix(self): #Generating Ergodic Markov Chain Matrix
        for i in range(self._size):
            a = [random.random() for _ in range(self._size)]
            sum_a = sum(a)
            b = list()
            for j in range(len(a)): #This loop is created in order all entrys in a row to sum up to 1.
                b.append(a[j]/sum_a) 
            self._matrix.append(np.array(b))
        self._matrix = np.array(self._matrix)

    def result_monitoring(self): #Monitoring the results of steady state operations.
        df1 = pd.DataFrame(self.matrix_multp(),columns = ["Steady State by Multp."])
        df2 = pd.DataFrame(self.mc_simulation(),columns = ["Steady State by MC"])
        df_merged = df1.merge(df2,left_index = True,right_index = True)
        df_merged["Diff."] = abs(df_merged["Steady State by Multp."] - df_merged["Steady State by MC"])
        mean_diff  = df_merged["Diff."].mean()
        print(f"*********************************************** Matrix Size = {self._size} ***********************************************")
        display(df_merged)
        print(f"Mean Numerical Difference = {mean_diff}")
    
    def matrix_multp(self):
        result = self._matrix

        while True:
            X_i = random.randint(0,self._size-1) #Choosing random state
            X_j = random.randint(0,self._size-1) #Choosing random entry for comparison

            difference = abs(np.array([result[i][X_i] for i in range(self._size)]).mean() - result[X_j][X_i])
            result = np.matmul(result,result)

            if difference < self._epsilon:
                mean_result = np.array([np.array([result[j][i] for j in range(len(result)) ]).mean() for i in range(len(result))])
                #print(result)
                return mean_result
                break

    def mc_simulation(self): #Monte-Carlo Simulation 

        X_i = random.randint(0,self._size-1) #In order to obtain initial state.

        for _ in range(self._step_size):
            j = np.random.random()
            for x in range(len(self._matrix[X_i])):
                if self._matrix[X_i][:x].sum() < j <= self._matrix[X_i][:x+1].sum():
                    X_i = x
                    self._MC_Visits[f"State_{x}"] += 1
                    break
        
        return np.array([j / self._step_size for j in self._MC_Visits.values()])

    def absorbing_includer(self):
        self._MC_Visits = {f"State_{i}": 0 for i in  range(self._size)}
        X_i = random.randint(0,self._size - 1) #Randomly choosing absorbing state
        absorbing_row = np.array([0]* self._size)
        absorbing_row[X_i] = 1
        self._matrix[X_i] = absorbing_row
        print("******************************************** Absorbing Included ***********************************************")
        self.result_monitoring()

### Now, Let's create the objects of TPM_Matrix class with size of 5,25 and 50 and observe the calculations.

In [41]:
Matrix_5 = TPM_Matrix(5)
Matrix_25 = TPM_Matrix(25)
Matrix_50 = TPM_Matrix(50)

Matrices = [Matrix_5,Matrix_25,Matrix_50]

for i in Matrices:
    i.generate_matrix()
    i.result_monitoring()
    i.absorbing_includer()

*********************************************** Matrix Size = 5 ***********************************************


,Steady State by Multp.,Steady State by MC,Diff.
0,0.200601,0.199935,0.000666
1,0.125283,0.124900,0.000383
2,0.178260,0.179610,0.001350
3,0.235933,0.236330,0.000397
4,0.259923,0.259225,0.000698


Mean Numerical Difference = 0.0006987369097814361
******************************************** Absorbing Included ***********************************************
*********************************************** Matrix Size = 5 ***********************************************


,Steady State by Multp.,Steady State by MC,Diff.
0,8.543004e-10,0.000000,8.543004e-10
1,5.332607e-10,0.000000,5.332607e-10
2,8.034228e-10,0.000005,4.999197e-06
3,1.492666e-09,0.000015,1.499851e-05
4,1.000000e+00,0.999980,1.999632e-05


Mean Numerical Difference = 7.999081564601255e-06
*********************************************** Matrix Size = 25 ***********************************************


...

Mean Numerical Difference = 0.0008986347560881814
*********************************************** Matrix Size = 50 ***********************************************


,Steady State by Multp.,Steady State by MC,Diff.
0,0.020272,0.019790,0.000482
1,0.020365,0.020325,0.000040
2,0.017135,0.017265,0.000130
3,0.019872,0.019730,0.000142
4,0.019143,0.019045,0.000098
5,0.020418,0.020450,0.000032
6,0.020752,0.020715,0.000037
7,0.018841,0.018620,0.000221
8,0.020076,0.020125,0.000049
9,0.019930,0.019860,0.000070


Mean Numerical Difference = 0.00022163471556881358
******************************************** Absorbing Included ***********************************************
*********************************************** Matrix Size = 50 ***********************************************


,Steady State by Multp.,Steady State by MC,Diff.
0,0.018439,0.0,0.018439
1,0.018877,0.0,0.018877
2,0.015669,0.0,0.015669
3,0.017859,0.0,0.017859
4,0.017806,0.0,0.017806
5,0.018560,0.0,0.018560
6,0.019213,0.0,0.019213
7,0.017518,0.0,0.017518
8,0.018106,0.0,0.018106
9,0.018655,0.0,0.018655


Mean Numerical Difference = 0.03586130402229419


### After observing the numerical approximations and differences, it will be beneficial to observe the efficiency of methods in terms of processing time.

In [14]:
size_5 = TPM_Matrix(5)
size_25 = TPM_Matrix(25)
size_50 = TPM_Matrix(50)

matrix_array = [size_5,size_25,size_50]
time_multp = []
time_simul = []

for i in matrix_array:
    i.generate_matrix()
    start = time.time()
    i.matrix_multp()
    intermediary = time.time()
    i.mc_simulation()
    end = time.time()
    
    matrix_time = intermediary - start
    simul_time = end - intermediary
    
    time_multp.append(matrix_time)
    time_simul.append(simul_time)

time_multp = np.array(time_multp)
time_simul = np.array(time_simul)


df1 = pd.DataFrame(time_multp, columns = ["Matrix Multp."])
df2 = pd.DataFrame(time_simul, columns = ["Monte Carlo Simul"])
df_merged = df1.merge(df2,left_index = True, right_index = True)
df_merged["Diff"] = df2["Monte Carlo Simul"] - df1["Matrix Multp."]
display(df_merged)

,Matrix Multp.,Monte Carlo Simul,Diff
0,0.000790,2.709408,2.708618
1,0.000560,10.357597,10.357037
2,0.002127,20.471201,20.469074


# Conclusions

** In terms of numerical differences,